In [1]:
# -*- coding = utf-8 -*-
"""
Main function to build recommendation systems.
"""
import utils
from ItemCF import ItemBasedCF
from LFM import LFM
from BasicSVD import BasicSVD
from UserCF import UserBasedCF
from dataset import DataSet
from most_popular import MostPopular
from random_pred import RandomPredict
from utils import LogTime


def run_model(model_name, dataset_name, test_size=0.3, clean=False, user_list = [1, 100, 233, 666, 888]):
    print('*' * 70)
    print('\tThis is %s model trained on %s with test_size = %.2f' % (model_name, dataset_name, test_size))
    print('*' * 70 + '\n')
    model_manager = utils.ModelManager(dataset_name, test_size)
    try:
        trainset = model_manager.load_model('trainset')
        testset = model_manager.load_model('testset')
    except OSError:
        ratings = DataSet.load_dataset(name=dataset_name)
        trainset, testset = DataSet.train_test_split(ratings, test_size=test_size)
        model_manager.save_model(trainset, 'trainset')
        model_manager.save_model(testset, 'testset')
    '''Do you want to clean workspace and retrain model again?'''
    '''if you want to change test_size or retrain model, please set clean_workspace True'''
    model_manager.clean_workspace(clean)
    if model_name == 'UserCF':
        model = UserBasedCF()
    elif model_name == 'ItemCF':
        model = ItemBasedCF()
    elif model_name == 'Random':
        model = RandomPredict()
    elif model_name == 'MostPopular':
        model = MostPopular()
    elif model_name == 'UserCF-IIF':
        model = UserBasedCF(use_iif_similarity=True)
    elif model_name == 'ItemCF-IUF':
        model = ItemBasedCF(use_iuf_similarity=True)
    elif model_name == 'LFM':
        # K, epochs, alpha, lamb, n_rec_movie
        model = LFM(10, 20, 0.1, 0.01, 10)
    elif model_name == 'BasicSVD':
        # K, epochs, alpha, lamb, n_rec_movie
        model = BasicSVD(10, 20, 0.1, 0.01, 10)
    else:
        raise ValueError('No model named ' + model_name)
    model.fit(trainset)
    model.test(testset)
    rec_result = recommend_test(model, user_list)
    return rec_result



def recommend_test(model, user_list):
    rec_result = {}
    for user in user_list:
        recommend = model.recommend(str(user))
        print("recommend for userid = %s:" % user)
        print(recommend)
        rec_result[user] = recommend
    return rec_result
main_time = LogTime(words="Main Function")
dataset_name = 'ml-100k'

model_type = 'LFM'
test_size = 0.1
rec_result = run_model(model_type, dataset_name, test_size, False, [1])

**********************************************************************
	This is LFM model trained on ml-100k with test_size = 0.10
**********************************************************************

LFM start...

User origin similarity model has saved before.
Load model success...

Test recommendation system start...
 steps(0), 0.01 seconds have spent..
Test recommendation system success.
total  step number is 943
total 2.08 seconds have spent

precision=0.2038	recall=0.1915	coverage=0.3010	popularity=5.2464

recommend for userid = 1:
['222', '168', '403', '568', '475', '385', '405', '550', '4', '431']


In [2]:
import json
file_path = "data/user_movie_feature/movies.csv"  # 请替换为实际的文件路径
dic_movie = {} # 用于存储解析后的数据
with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        # 解析JSON对象
        record = json.loads(line.strip())
#         data.append(record)
#         print(record)
        dic_movie[str(record['id'])] = record['title']

In [3]:
rec_movie_name = [dic_movie[x] for x in rec_result[1]]
print(rec_movie_name)

['Star Trek: First Contact (1996)', 'Monty Python and the Holy Grail (1974)', 'Batman (1989)', 'Speed (1994)', 'Trainspotting (1996)', 'True Lies (1994)', 'Mission: Impossible (1996)', 'Die Hard: With a Vengeance (1995)', 'Get Shorty (1995)', 'Highlander (1986)']


In [4]:
import pickle
with open('movie_url_pic.pickle', 'rb') as f:
        movie_data = pickle.load(f)
dic = {}
for name in rec_movie_name:
    if name in movie_data:
        dic[name] = movie_data[name]
        with open('show_movie_url_pic.pickle', 'wb') as f:
            pickle.dump(dic, f)
with open('show_movie_url_pic.pickle', 'rb') as f:
        show_movie_data = pickle.load(f)
print(show_movie_data)

{'Star Trek: First Contact (1996)': ['https://www.imdb.com/title/tt0117731/?ref_=fn_al_tt_1', './pictures/Star Trek: First Contact (1996).jpg'], 'Batman (1989)': ['https://www.imdb.com/title/tt0096895/?ref_=fn_al_tt_1', './pictures/Batman (1989).jpg'], 'Speed (1994)': ['https://www.imdb.com/title/tt0111257/?ref_=fn_al_tt_1', './pictures/Speed (1994).jpg'], 'Trainspotting (1996)': ['https://www.imdb.com/title/tt0117951/?ref_=fn_al_tt_1', './pictures/Trainspotting (1996).jpg'], 'True Lies (1994)': ['https://www.imdb.com/title/tt0111503/?ref_=fn_al_tt_1', './pictures/True Lies (1994).jpg'], 'Mission: Impossible (1996)': ['https://www.imdb.com/title/tt0117060/?ref_=fn_al_tt_1', './pictures/Mission: Impossible (1996).jpg'], 'Die Hard: With a Vengeance (1995)': ['https://www.imdb.com/title/tt0112864/?ref_=fn_al_tt_1', './pictures/Die Hard: With a Vengeance (1995).jpg'], 'Get Shorty (1995)': ['https://www.imdb.com/title/tt0113161/?ref_=fn_al_tt_1', './pictures/Get Shorty (1995).jpg'], 'Highla

In [ ]:
!python3 main2.py

 * Serving Flask app 'main2' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 683-707-298
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET /static/pictures/Batman%20%281989%29.jpg HTTP/1.1" 304 -
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET /static/pictures/Star%20Trek:%20First%20Contact%20%281996%29.jpg HTTP/1.1" 304 -
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET /static/pictures/Trainspotting%20%281996%29.jpg HTTP/1.1" 304 -
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET /static/pictures/Speed%20%281994%29.jpg HTTP/1.1" 304 -
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET /static/pictures/True%20Lies%20%281994%29.jpg HTTP/1.1" 304 -
127.0.0.1 - - [11/Apr/2024 22:57:47] "GET /static/pictures/Mission:%20Impossible%20%281996%29.jpg HTTP/1.1" 304 -
127.0.0.1 - - [11/Apr/2024 22:

In [4]:
rec_result[1]

['222', '168', '403', '568', '475', '385', '405', '550', '4', '431']